NCDC CDO web services token: TTyjEBwflgZUPriqiAXQRZrnCgcfPgxD

NWS

In [1]:
import requests

start_date = "2023-01-01"
end_date = "2023-04-30"
location_id = "ZIP:10001"  # 紐約市的郵政編碼，這裡以 10001 為例

url = f'https://api.weather.gov/points/38.8894,-77.0352'

response = requests.get(url)

data = response.json()
    
url2 = data['properties']['forecast']
print(url2)

response = requests.get(url2)

if response.status_code == 200:
    data = response.json()
    
    periods = data["properties"]["periods"]
    
    for period in periods:
        name = period["name"]
        startTime = period["startTime"]
        endTime = period["endTime"]
        temperature = period["temperature"]
        temperature_unit = period["temperatureUnit"]
        detailed_forecast = period["detailedForecast"]
        
        # 檢查詳細預報是否包含降雨相關的字詞
        if "rain" in detailed_forecast.lower() or "showers" in detailed_forecast.lower():
            has_rain = True
        else:
            has_rain = False
        
        print(f"Name: {name}")
        print(f"Start Time: {startTime}")
        print(f"End Time: {endTime}")
        print(f"Temperature: {temperature} {temperature_unit}")
        print(f"Detailed Forecast: {detailed_forecast}")
        print(f"Has Rain: {has_rain}")
        print("-------------------")
        
else:
    print("API request failed.")

https://api.weather.gov/gridpoints/LWX/97,71/forecast
Name: Overnight
Start Time: 2023-07-11T05:00:00-04:00
End Time: 2023-07-11T06:00:00-04:00
Temperature: 69 F
Detailed Forecast: Mostly clear, with a low around 69. West wind around 3 mph.
Has Rain: False
-------------------
Name: Tuesday
Start Time: 2023-07-11T06:00:00-04:00
End Time: 2023-07-11T18:00:00-04:00
Temperature: 92 F
Detailed Forecast: Sunny, with a high near 92. Northwest wind around 5 mph.
Has Rain: False
-------------------
Name: Tuesday Night
Start Time: 2023-07-11T18:00:00-04:00
End Time: 2023-07-12T06:00:00-04:00
Temperature: 72 F
Detailed Forecast: Mostly clear, with a low around 72. Southwest wind around 5 mph.
Has Rain: False
-------------------
Name: Wednesday
Start Time: 2023-07-12T06:00:00-04:00
End Time: 2023-07-12T18:00:00-04:00
Temperature: 94 F
Detailed Forecast: Sunny, with a high near 94. West wind 3 to 8 mph.
Has Rain: False
-------------------
Name: Wednesday Night
Start Time: 2023-07-12T18:00:00-04:00


# OpenWeather

key: eefecb7998d70140ab925260f716b4e1

## Current weather data

In [2]:
import requests
import json

key = 'eefecb7998d70140ab925260f716b4e1'
# 經緯度只能抓到小數點後4位
lat = 40.757306
lon = -73.974518
units = 'metric'  # 公制(攝氏)

url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={key}&units={units}'

response = requests.get(url)
print(f'Status: {response.status_code}')
data_current = response.json()
print(data_current)

Status: 200
{'coord': {'lon': -73.9745, 'lat': 40.7573}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 20.04, 'feels_like': 19.8, 'temp_min': 15.24, 'temp_max': 22.27, 'pressure': 1012, 'humidity': 65}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 320}, 'clouds': {'all': 0}, 'dt': 1689070519, 'sys': {'type': 2, 'id': 2003558, 'country': 'US', 'sunrise': 1689068044, 'sunset': 1689121695}, 'timezone': -14400, 'id': 5125771, 'name': 'Manhattan', 'cod': 200}


In [3]:
for i in data_current:
    print(data_current[i])

{'lon': -73.9745, 'lat': 40.7573}
[{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]
stations
{'temp': 20.04, 'feels_like': 19.8, 'temp_min': 15.24, 'temp_max': 22.27, 'pressure': 1012, 'humidity': 65}
10000
{'speed': 3.09, 'deg': 320}
{'all': 0}
1689070519
{'type': 2, 'id': 2003558, 'country': 'US', 'sunrise': 1689068044, 'sunset': 1689121695}
-14400
5125771
Manhattan
200


In [4]:
for i in data_current['main']:
    print(str(i) + ': ', data_current['main'][i])

temp:  20.04
feels_like:  19.8
temp_min:  15.24
temp_max:  22.27
pressure:  1012
humidity:  65


# Geocoding API

取得紐約經緯度

In [5]:
import requests
key = 'eefecb7998d70140ab925260f716b4e1'
city = 'New York'
state = 'NY'
country = 'US'


url = f'http://api.openweathermap.org/geo/1.0/direct?q={city},{state},{country}&appid={key}'


response = requests.get(url)
print(f'Status: {response.status_code}')
data_geo = response.json()

ny_lat = data_geo[0]['lat']
ny_lon = data_geo[0]['lon']
print(f'New York latitude: {ny_lat}')
print(f'New York longitude: {ny_lon}')

Status: 200
New York latitude: 40.7127281
New York longitude: -74.0060152


## 5 day weather forecast

未來5天(含今天)的逐3小時預報

In [6]:
import requests

key = 'eefecb7998d70140ab925260f716b4e1'
# 經緯度只能抓到小數點後4位
lat = ny_lat
lon = ny_lon
units = 'metric'  # 公制(攝氏)
lang = 'zh_tw'
timezone = 'GMT-4'


url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key}&units={units}&lang={lang}&timezone=America/New_York'

response = requests.get(url)
print(f'Status: {response.status_code}')
data_5day = response.json()
print(data_5day)

Status: 200
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1689076800, 'main': {'temp': 21.18, 'feels_like': 20.95, 'temp_min': 21.18, 'temp_max': 23.19, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 1012, 'humidity': 61, 'temp_kf': -2.01}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 3.2, 'deg': 288, 'gust': 4.96}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2023-07-11 12:00:00'}, {'dt': 1689087600, 'main': {'temp': 25.49, 'feels_like': 25.4, 'temp_min': 25.49, 'temp_max': 28.15, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 1012, 'humidity': 50, 'temp_kf': -2.66}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 3.64, 'deg': 292, 'gust': 4.79}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2023-07-11 15:00:00'}, {'dt': 1689098400, 'main': {'temp': 31.14, 'feels_like': 30.42, 'temp_min': 31.

In [7]:
# for i in range(40):
#     print(data_5day['list'][i]['dt_txt'])
#     i += 1
print(data_5day['list'][0]['dt_txt'])

2023-07-11 12:00:00


存成DataFrame

In [8]:
import pandas as pd

dt = {'date_time': [data_5day['list'][i]['dt_txt'] for i in range(40)]}

df_weather_5day = pd.DataFrame(dt)

In [9]:
def add_col_temp(df, feature_name, key):
    '''
    Extracts temperature data from the 'main' section of the JSON file returned by the 5-day weather forecast API,
    and stores it in the specified DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame where the temperature data will be stored.
        col_name (str): The name of the column in the DataFrame to store the temperature data.
        key (str): The specific temperature item to extract from the 'main' section.
    Note:
        This function does not return a value. The results generated will be directly added to the specified DataFrame.
    '''
    new_col = {feature_name: [data_5day['list'][i]['main'][key] for i in range(40)]}
    df[feature_name] = pd.DataFrame(new_col)


add_col_temp(df_weather_5day, 'temp', 'temp')
add_col_temp(df_weather_5day, 'body_temp', 'feels_like')
add_col_temp(df_weather_5day, 'temp_min', 'temp_min')
add_col_temp(df_weather_5day, 'temp_max', 'temp_max')

df_weather_5day.head()

,date_time,temp,body_temp,temp_min,temp_max
0,2023-07-11 12:00:00,21.18,20.95,21.18,23.19
1,2023-07-11 15:00:00,25.49,25.40,25.49,28.15
2,2023-07-11 18:00:00,31.14,30.42,31.14,31.14
3,2023-07-11 21:00:00,31.70,31.38,31.70,31.70
4,2023-07-12 00:00:00,29.44,29.61,29.44,29.44


In [10]:
# weather: Take main, description

def add_col_weather(df, feature_name, key):
    '''
    Extracts temperature data from the 'main' section of the JSON file returned by the 5-day weather forecast API,
    and stores it in the specified DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame where the temperature data will be stored.
        col_name (str): The name of the column in the DataFrame to store the temperature data.
        key (str): The specific temperature item to extract from the 'main' section.
    Note:
        This function does not return a value. The results generated will be directly added to the specified DataFrame.
    '''
    new_col = {feature_name: [data_5day['list'][i]['weather'][0][key] for i in range(40)]}
    df[feature_name] = pd.DataFrame(new_col)

add_col_weather(df_weather_5day, 'weather_condition', 'main')
add_col_weather(df_weather_5day, 'weather_description', 'description')

df_weather_5day.head(20)

,date_time,temp,body_temp,temp_min,temp_max,weather_condition,weather_description
0,2023-07-11 12:00:00,21.18,20.95,21.18,23.19,Clear,晴
1,2023-07-11 15:00:00,25.49,25.40,25.49,28.15,Clear,晴
2,2023-07-11 18:00:00,31.14,30.42,31.14,31.14,Clear,晴
3,2023-07-11 21:00:00,31.70,31.38,31.70,31.70,Clear,晴
4,2023-07-12 00:00:00,29.44,29.61,29.44,29.44,Clear,晴
5,2023-07-12 03:00:00,26.85,27.64,26.85,26.85,Clear,晴
6,2023-07-12 06:00:00,24.99,25.06,24.99,24.99,Clear,晴
7,2023-07-12 09:00:00,23.76,23.76,23.76,23.76,Clear,晴
8,2023-07-12 12:00:00,25.49,25.53,25.49,25.49,Clear,晴
9,2023-07-12 15:00:00,30.76,30.75,30.76,30.76,Clear,晴


In [11]:
print(data_5day['city']['timezone'])

-14400


In [14]:
from datetime import datetime
dt = datetime.fromtimestamp(data_5day['list'][0]['dt'])
print(data_5day['list'][0]['dt'])
print(dt)

1689076800
2023-07-11 20:00:00


In [31]:
from datetime import datetime
from datetime import timedelta


dt = datetime.strptime(data_5day['list'][0]['dt_txt'], '%Y-%m-%d %H:%M:%S')
print(dt)
td = timedelta(hours=4)
print(dt - td)

2023-07-11 12:00:00
2023-07-11 08:00:00


原資料直接取出會是UTC標準時間

In [37]:
from datetime import datetime
from datetime import timedelta

lst = []
for i in range(40):
    dt_utc = data_5day['list'][i]['dt_txt']
    td = timedelta(hours=4)
    dt_ny = datetime.strptime(dt_utc, '%Y-%m-%d %H:%M:%S') - td
    lst.append(dt_ny)

dt_dict = {'dt': lst}

df_weather_5day['date_time'] = pd.DataFrame(dt_dict)
df_weather_5day.head(20)


,date_time,temp,body_temp,temp_min,temp_max,weather_condition,weather_description,dt
0,2023-07-11 08:00:00,21.18,20.95,21.18,23.19,Clear,晴,2023-07-11 08:00:00
1,2023-07-11 11:00:00,25.49,25.40,25.49,28.15,Clear,晴,2023-07-11 11:00:00
2,2023-07-11 14:00:00,31.14,30.42,31.14,31.14,Clear,晴,2023-07-11 14:00:00
3,2023-07-11 17:00:00,31.70,31.38,31.70,31.70,Clear,晴,2023-07-11 17:00:00
4,2023-07-11 20:00:00,29.44,29.61,29.44,29.44,Clear,晴,2023-07-11 20:00:00
5,2023-07-11 23:00:00,26.85,27.64,26.85,26.85,Clear,晴,2023-07-11 23:00:00
6,2023-07-12 02:00:00,24.99,25.06,24.99,24.99,Clear,晴,2023-07-12 02:00:00
7,2023-07-12 05:00:00,23.76,23.76,23.76,23.76,Clear,晴,2023-07-12 05:00:00
8,2023-07-12 08:00:00,25.49,25.53,25.49,25.49,Clear,晴,2023-07-12 08:00:00
9,2023-07-12 11:00:00,30.76,30.75,30.76,30.76,Clear,晴,2023-07-12 11:00:00
